#### Includes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import altair as alt
from altair import datum
import pandas as pd
import time

In [3]:
import sources
from steered_cnn.models import *

## Init

In [4]:
train, val, test = sources.load_dataset()

/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
net = UNet(6, nfeatures_base=32).cuda()

## Run Model

In [6]:
for data in train:
    r = net(x=data['x'].cuda())
    break

In [7]:
r.shape

torch.Size([8, 1, 560, 560])